In [1]:
a = dict([ (1,2) , (22,3),(33,22) ])

In [33]:
a

{1: 2, 22: 3, 33: 22}

In [2]:
from itertools import combinations

In [44]:
print(list((combinations([1,2,3], 2))))

[(1, 2), (1, 3), (2, 3)]


In [93]:
next(zip([1],[3]))

(1, 3)

In [3]:
embeddings = dict(zip([1,2, 3] , [np.array([12, -22]), np.array([-3,-3]), np.array([1,-13]) ] ))

NameError: name 'np' is not defined

In [36]:
embeddings

{1: array([ 12, -22]), 2: array([-3, -3]), 3: array([  1, -13])}

In [37]:
(list(embeddings.keys()),

                     dict(zip(tuple(combinations(embeddings.keys(), 2)),
                              
                              [round(cosine(embeddings[x[0]], embeddings[x[1]]),8) for x in
                               
                               list(combinations(embeddings.keys(), 2))])))

([1, 2, 3], {(1, 2): 0.71783368, (1, 3): 0.087964, (2, 3): 0.34920863})

In [20]:
a

{1: 2, 22: 3, 33: 22}

In [38]:
tuple(combinations(a.keys(), 2))

((1, 22), (1, 33), (22, 33))

In [44]:
a[list(combinations(a.keys(),2))[0][1]]

3

In [47]:
d1 = dict(zip(tuple(combinations(a.keys(), 2)),  
              [cosine([a[x[0]]], [a[x[1]]]) for x in list(combinations(a.keys(), 2) ) ]))

In [312]:
list(np.unique(list(d1.keys())))

array([1, 2, 3])

In [48]:
d1.keys()

dict_keys([(1, 22), (1, 33), (22, 33)])

In [49]:
d1

{(1, 22): 0, (1, 33): 0, (22, 33): 0}

In [12]:
d1

NameError: name 'd1' is not defined

In [50]:
sim = d1

In [369]:
asd

array([1, 2, 3])

In [396]:
sim

{(1, 2): 0.7178336760084498,
 (1, 3): 0.0879639993170358,
 (2, 3): 0.3492086265440315}

In [53]:
set(sim) - set([1])

{(1, 22), (1, 33), (22, 33)}

In [52]:
set([1,3]) & set([1,2]) 

{1}

In [51]:
knn_dict = dict()
for x in sim:
    key = (set([x]) - set(knn_dict)).pop()
    knn_dict[key] ={ ((set([x]) & set([k]))).pop() : sim[k] for k in sim if len(set([x]) - set([k])) == 1 }

KeyError: 'pop from an empty set'

In [25]:
knn_dict

{}

In [ ]:
{1: [(2, 0.7178336760084498), (3, 0.0879639993170358)],
 2: [(1, 0.7178336760084498), (3, 0.3492086265440315)],
 3: [(2, 0.3492086265440315), (1, 0.0879639993170358)]}

In [26]:
knn_dict

{}

In [427]:
knn_dict

{(2, 3): (1, 3)}

In [419]:
knn_dict

set()

In [54]:
knn_dict = dict()
for x in sim:
    knn_dict = {( (set([x]) - set([k])) , sim[k]) for k in sim if len(set([x]) - set([k])) > 0 }
#knn_dict = {int(x): sorted(knn_dict[x][:2], 
#                      key=lambda x :x[1], reverse = True) for x in knn_dict}

TypeError: unhashable type: 'set'

In [55]:
knn_dict

{}

[1, 22, 33]

In [63]:
prices

{1: 5.123123, 22: 20.22, 33: 2.333}

In [274]:
[x[1] for x in knn_dict[1]]

[0.7178336760084498, 0.0879639993170358]

In [24]:
"""Solution for Similar Items task"""
from typing import Dict
from typing import List
from typing import Tuple
from itertools import combinations
import numpy as np
from scipy.spatial.distance import cosine


class SimilarItems:
    """Similar items class"""

    @staticmethod
    def similarity(embeddings: Dict[int, np.ndarray]) -> Dict[Tuple[int, int], float]:
        """Calculate pairwise similarities between each item
        in embedding.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings.

        Returns:
            Tuple[List[str], Dict[Tuple[int, int], float]]:
            List of all items + Pairwise similarities dict
            Keys are in form of (i, j) - combinations pairs of item_ids
            with i < j.
            Round each value to 8 decimal places.
        """

        pair_sims = dict(zip(tuple(combinations(embeddings.keys(), 2)),
                             [round(1 - cosine(embeddings[x[0]], embeddings[x[1]]), 8) for x in
                              list(combinations(embeddings.keys(), 2))]))

        return pair_sims

    @staticmethod
    def knn(
        sim: Dict[Tuple[int, int], float], top: int
    ) -> Dict[int, List[Tuple[int, float]]]:
        """Return closest neighbors for each item.

        Args:
            sim (Dict[Tuple[int, int], float]): <similarity> method output.
            top (int): Number of top neighbors to consider.

        Returns:
            Dict[int, List[Tuple[int, float]]]: Dict with top closest neighbors
            for each item.
        """

        knn_dict = dict()
        asd = np.unique(list(sim.keys()))

        for i in asd:
            a_vect = np.array(list(sim.keys()))
            ind = list(np.where(np.any(a_vect == i, axis=1))[0])
            val = np.array(list(sim.values()))[ind]

            knn_dict[i] = list(
                zip(list(a_vect[ind][a_vect[ind] != i]), list(val)))
            
        knn_dict = {int(x): sorted(knn_dict[x][:top],
                                   key=lambda x: x[1], reverse=True) for x in knn_dict}

        return knn_dict

    @staticmethod
    def knn_price(
        knn_dict: Dict[int, List[Tuple[int, float]]],
        prices: Dict[int, float],
    ) -> Dict[int, float]:
        """Calculate weighted average prices for each item.
        Weights should be positive numbers in [0, 2] interval.

        Args:
            knn_dict (Dict[int, List[Tuple[int, float]]]): <knn> method output.
            prices (Dict[int, float]): Price dict for each item.

        Returns:
            Dict[int, float]: New prices dict, rounded to 2 decimal places.
        """
        prices_new = dict()
        for x in prices:
            list_ = [k[0] for k in knn_dict[x]]
            norm = np.array([k[1] + 1 for k in knn_dict[x]])
            prices_new[x] = round(sum(np.array([prices[k] for
                                            k in list_]) * norm)/sum(norm), 2)

        return prices_new

    @staticmethod
    def transform(
        embeddings: Dict[int, np.ndarray],
        prices: Dict[int, float],
        top: int,
    ) -> Dict[int, float]:
        """Transforming input embeddings into a dictionary
        with weighted average prices for each item.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings.
            prices (Dict[int, float]): Price dict for each item.
            top (int): Number of top neighbors to consider.

        Returns:
            Dict[int, float]: Dict with weighted average prices for each item.
        """
        knn_price_dict = SimilarItems().similarity(embeddings)
        knn_price_dict = SimilarItems().knn(knn_price_dict, top)
        knn_price_dict = SimilarItems().knn_price(knn_price_dict, prices)

        return knn_price_dict

In [5]:
k= 30
embeddings = dict(zip([i for i in range(0, k)], [np.array([12, -22]), np.array([-3,-3]), 
                                       np.array([1,-13])]* k))

In [6]:
embeddings

{0: array([ 12, -22]),
 1: array([-3, -3]),
 2: array([  1, -13]),
 3: array([ 12, -22]),
 4: array([-3, -3]),
 5: array([  1, -13]),
 6: array([ 12, -22]),
 7: array([-3, -3]),
 8: array([  1, -13]),
 9: array([ 12, -22]),
 10: array([-3, -3]),
 11: array([  1, -13]),
 12: array([ 12, -22]),
 13: array([-3, -3]),
 14: array([  1, -13]),
 15: array([ 12, -22]),
 16: array([-3, -3]),
 17: array([  1, -13]),
 18: array([ 12, -22]),
 19: array([-3, -3]),
 20: array([  1, -13]),
 21: array([ 12, -22]),
 22: array([-3, -3]),
 23: array([  1, -13]),
 24: array([ 12, -22]),
 25: array([-3, -3]),
 26: array([  1, -13]),
 27: array([ 12, -22]),
 28: array([-3, -3]),
 29: array([  1, -13])}

In [27]:
prices = dict(zip(list(embeddings.keys()), [5.123123,20.22,2.333] * k))

In [9]:
a = SimilarItems().similarity(embeddings)

In [10]:
a

{(0, 1): 0.28216632,
 (0, 2): 0.912036,
 (0, 3): 1,
 (0, 4): 0.28216632,
 (0, 5): 0.912036,
 (0, 6): 1,
 (0, 7): 0.28216632,
 (0, 8): 0.912036,
 (0, 9): 1,
 (0, 10): 0.28216632,
 (0, 11): 0.912036,
 (0, 12): 1,
 (0, 13): 0.28216632,
 (0, 14): 0.912036,
 (0, 15): 1,
 (0, 16): 0.28216632,
 (0, 17): 0.912036,
 (0, 18): 1,
 (0, 19): 0.28216632,
 (0, 20): 0.912036,
 (0, 21): 1,
 (0, 22): 0.28216632,
 (0, 23): 0.912036,
 (0, 24): 1,
 (0, 25): 0.28216632,
 (0, 26): 0.912036,
 (0, 27): 1,
 (0, 28): 0.28216632,
 (0, 29): 0.912036,
 (1, 2): 0.65079137,
 (1, 3): 0.28216632,
 (1, 4): 1,
 (1, 5): 0.65079137,
 (1, 6): 0.28216632,
 (1, 7): 1,
 (1, 8): 0.65079137,
 (1, 9): 0.28216632,
 (1, 10): 1,
 (1, 11): 0.65079137,
 (1, 12): 0.28216632,
 (1, 13): 1,
 (1, 14): 0.65079137,
 (1, 15): 0.28216632,
 (1, 16): 1,
 (1, 17): 0.65079137,
 (1, 18): 0.28216632,
 (1, 19): 1,
 (1, 20): 0.65079137,
 (1, 21): 0.28216632,
 (1, 22): 1,
 (1, 23): 0.65079137,
 (1, 24): 0.28216632,
 (1, 25): 1,
 (1, 26): 0.65079137,
 (

In [11]:
sim = a

In [67]:
knn_dict = dict()
asd = np.unique(list(sim.keys()))

for i in asd:
    a_vect = np.array(list(sim.keys()))
    ind = list(np.where(np.any(a_vect == i, axis=1))[0])
    val = np.array(list(sim.values()))[ind]
    
    knn_dict[i] = list(
        zip(list(a_vect[ind][a_vect[ind] != i]), list(val)))
knn_dict = {int(x): sorted(knn_dict[x],
                           key=lambda x: x[1], reverse=True)[:2] for x in knn_dict}

In [42]:
a_vect[434]

array([28, 29])

In [68]:
knn_dict

{0: [(3, 1.0), (6, 1.0)],
 1: [(4, 1.0), (7, 1.0)],
 2: [(5, 1.0), (8, 1.0)],
 3: [(0, 1.0), (6, 1.0)],
 4: [(1, 1.0), (7, 1.0)],
 5: [(2, 1.0), (8, 1.0)],
 6: [(0, 1.0), (3, 1.0)],
 7: [(1, 1.0), (4, 1.0)],
 8: [(2, 1.0), (5, 1.0)],
 9: [(0, 1.0), (3, 1.0)],
 10: [(1, 1.0), (4, 1.0)],
 11: [(2, 1.0), (5, 1.0)],
 12: [(0, 1.0), (3, 1.0)],
 13: [(1, 1.0), (4, 1.0)],
 14: [(2, 1.0), (5, 1.0)],
 15: [(0, 1.0), (3, 1.0)],
 16: [(1, 1.0), (4, 1.0)],
 17: [(2, 1.0), (5, 1.0)],
 18: [(0, 1.0), (3, 1.0)],
 19: [(1, 1.0), (4, 1.0)],
 20: [(2, 1.0), (5, 1.0)],
 21: [(0, 1.0), (3, 1.0)],
 22: [(1, 1.0), (4, 1.0)],
 23: [(2, 1.0), (5, 1.0)],
 24: [(0, 1.0), (3, 1.0)],
 25: [(1, 1.0), (4, 1.0)],
 26: [(2, 1.0), (5, 1.0)],
 27: [(0, 1.0), (3, 1.0)],
 28: [(1, 1.0), (4, 1.0)],
 29: [(2, 1.0), (5, 1.0)]}

In [61]:
a_vect[ind][a_vect[ind] != 29]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [59]:
np.array(list(sim.values()))[ind]

array([0.912036  , 0.65079137, 1.        , 0.912036  , 0.65079137,
       1.        , 0.912036  , 0.65079137, 1.        , 0.912036  ,
       0.65079137, 1.        , 0.912036  , 0.65079137, 1.        ,
       0.912036  , 0.65079137, 1.        , 0.912036  , 0.65079137,
       1.        , 0.912036  , 0.65079137, 1.        , 0.912036  ,
       0.65079137, 1.        , 0.912036  , 0.65079137])

In [45]:
ind = list(np.where(np.any(a_vect == 29, axis=1))[0])

In [17]:
knn_dict

{0: [(2, 0.912036), (1, 0.28216632)],
 1: [(2, 0.65079137), (0, 0.28216632)],
 2: [(0, 0.912036), (1, 0.65079137)],
 3: [(0, 1.0), (1, 0.28216632)],
 4: [(1, 1.0), (0, 0.28216632)],
 5: [(0, 0.912036), (1, 0.65079137)],
 6: [(0, 1.0), (1, 0.28216632)],
 7: [(1, 1.0), (0, 0.28216632)],
 8: [(0, 0.912036), (1, 0.65079137)],
 9: [(0, 1.0), (1, 0.28216632)],
 10: [(1, 1.0), (0, 0.28216632)],
 11: [(0, 0.912036), (1, 0.65079137)],
 12: [(0, 1.0), (1, 0.28216632)],
 13: [(1, 1.0), (0, 0.28216632)],
 14: [(0, 0.912036), (1, 0.65079137)],
 15: [(0, 1.0), (1, 0.28216632)],
 16: [(1, 1.0), (0, 0.28216632)],
 17: [(0, 0.912036), (1, 0.65079137)],
 18: [(0, 1.0), (1, 0.28216632)],
 19: [(1, 1.0), (0, 0.28216632)],
 20: [(0, 0.912036), (1, 0.65079137)],
 21: [(0, 1.0), (1, 0.28216632)],
 22: [(1, 1.0), (0, 0.28216632)],
 23: [(0, 0.912036), (1, 0.65079137)],
 24: [(0, 1.0), (1, 0.28216632)],
 25: [(1, 1.0), (0, 0.28216632)],
 26: [(0, 0.912036), (1, 0.65079137)],
 27: [(0, 1.0), (1, 0.28216632)],
 

In [30]:
%%time
SimilarItems().transform(embeddings, prices, top = 2)

CPU times: user 19.6 ms, sys: 0 ns, total: 19.6 ms
Wall time: 19.1 ms


{0: 9.51,
 1: 3.55,
 2: 12.12,
 3: 11.02,
 4: 14.32,
 5: 12.12,
 6: 11.02,
 7: 14.32,
 8: 12.12,
 9: 11.02,
 10: 14.32,
 11: 12.12,
 12: 11.02,
 13: 14.32,
 14: 12.12,
 15: 11.02,
 16: 14.32,
 17: 12.12,
 18: 11.02,
 19: 14.32,
 20: 12.12,
 21: 11.02,
 22: 14.32,
 23: 12.12,
 24: 11.02,
 25: 14.32,
 26: 12.12,
 27: 11.02,
 28: 14.32,
 29: 12.12}